# Obtención de los datos

In [1]:
# La primera parte del proyecto consiste en obtener los datos de distintas paginas web
# El objetivo principal es obtener el precio de la electricidad desde el 1 de enero de 2016 hasta el 30 de junio de 2024, 3104 días
# El precio de la electricidad cambia cada hora, por lo que en total se recogeran 3104*24=74.496 datos más o menos
# Aparte del precio, la idea es obtener más datos que puedan estar relacionados como la demanda y generación de energía.
# Para obtener los datos de los precios de la electricada y del gas se ha utilizado Web Scraping

In [3]:
# Importando las librerias
# -------------------------
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.chrome.service import Service

## Precio de la luz para cada hora

In [47]:
# Links de los precios de la electricidad para cada hora de España, Francia y Portugal
url_S = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2016+00:00|CET|DAY&biddingZone.values=CTY|10YES-REE------0!BZN|10YES-REE------0&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)"
url_P = 'https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2016+00:00|CET|DAY&biddingZone.values=CTY|10YPT-REN------W!BZN|10YPT-REN------W&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)'
url_F = 'https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2016+00:00|CET|DAY&biddingZone.values=CTY|10YFR-RTE------C!BZN|10YFR-RTE------C&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)'
response = requests.get(url_S)
print(bool(response))
soup = BeautifulSoup(response.text, "html.parser")
chrome_driver = "chromedriver.exe"
driver_service = Service(executable_path = chrome_driver)

True


In [ ]:
precio_S=[]
navegador = webdriver.Chrome(service=driver_service)
navegador.get(url_S)
sleep(4)
dias=3104
for j in range(dias):
    sleep(randint(2,3))
    soup = BeautifulSoup(navegador.page_source)

    for diru in soup.find_all('td', attrs={"class":"dv-value-cell"}):
        precio_S.append(diru.text)
    elemento=navegador.find_element(By.XPATH, '//*[@id="date-arrow-right-user-friendly-input-dv-date-from"]')
    elemento.click()

In [ ]:
precio_P=[]
navegador = webdriver.Chrome(service=driver_service)
navegador.get('https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=29.12.2021+00:00|CET|DAY&biddingZone.values=CTY|10YPT-REN------W!BZN|10YPT-REN------W&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)')
sleep(4)
for j in range(dias):
    sleep(randint(2,3))
    soup = BeautifulSoup(navegador.page_source)

    for diru in soup.find_all('td', attrs={"class":"dv-value-cell"}):
        precio_P.append(diru.text)
    elemento=navegador.find_element(By.XPATH, '//*[@id="date-arrow-right-user-friendly-input-dv-date-from"]')
    elemento.click()

In [ ]:
precio_F=[]
navegador = webdriver.Chrome(service=driver_service)
navegador.get('https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&biddingZone.values=CTY|10YFR-RTE------C!BZN|10YFR-RTE------C&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)')
sleep(10)
for j in range(dias):
    sleep(randint(2,3))
    soup = BeautifulSoup(navegador.page_source)

    for diru in soup.find_all('td', attrs={"class":"dv-value-cell"}):
        precio_F.append(diru.text)
    elemento=navegador.find_element(By.XPATH, '//*[@id="date-arrow-right-user-friendly-input-dv-date-from"]')
    elemento.click()

In [ ]:
# Lo guardamos todo en un dataset y lo convertimos en csv
df_PrecioLuz= pd.DataFrame(list(zip(hora,precio_P,precio_F,precio_S)), columns=['Hora','Precio_PT','Precio_FR','Precio_ES'])
df_PrecioLuz.to_csv('Precio_Luz_hora.csv', index=False)

## Generación, demanda y export/import. de energía eléctrica

In [ ]:
# La generación, demanda, importaciones de energía se consiguieron directamente de la REE 
# Desde este dataset se consiguió la lista de los días desde 206-01-01 a 2024-06-30

In [13]:
df_Balance=pd.read_csv('Balance_diario.csv')

In [15]:
Fecha=[]
for i in df_Balance['Fecha']:
    Fecha.append(i)

## Precio de la luz para cada día. Se ha hecho la media aritmética del dataset anterior

In [7]:
df_precio=pd.read_csv('Precio_Luz_hora.csv') # Hay que fijarse cuando toca el cambio de hora
df_precio.head()

,Hora,Precio_PT,Precio_FR,Precio_ES
0,2016-01-01 00:00:00,48.55,23.86,48.55
1,2016-01-01 01:00:00,40.00,22.39,40.00
2,2016-01-01 02:00:00,33.10,20.59,33.10
3,2016-01-01 03:00:00,28.11,16.81,28.11
4,2016-01-01 04:00:00,27.13,17.41,27.13


In [9]:
def hora_a_dia(lista):
    ''' Con esta función paso hago una media de los precios horarioa para ese día
    Es importante tener en cuenta los días con cambios de hora. Ese día solo hay 23 horas
    '''
    diario=[]
    for i in range(0,2064,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[2064:2087])/23) # 2016
                  
    for i in range(2087,10799,24):
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[10799:10822])/23) # 2017
                  
    for i in range(10822,19534,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[19534:19557])/23) # 2018
                  
    for i in range(19557,28437,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[28437:28460])/23) # 2019
                  
    for i in range(28460,37172,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[37172:37195])/23) # 2020
                  
    for i in range(37195,45907,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[45907:45930])/23) # 2021
                  
    for i in range(45930,54642,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[54642:54665])/23) # 2022
                  
    for i in range(54665,63377,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[63377:63400])/23) # 2023
                  
    for i in range(63400,72280,24): 
        diario.append(sum(lista[i:i+23])/24)
    diario.append(sum(lista[72280:72303])/23) # 2024
                  
    for i in range(72303,74487,24): 
        diario.append(sum(lista[i:i+23])/24)
    return diario

In [17]:
PT=hora_a_dia(df_precio.Precio_PT)
FR=hora_a_dia(df_precio.Precio_FR)
ES=hora_a_dia(df_precio.Precio_ES)

In [19]:
df_Precio_luz_diario = pd.DataFrame(list(zip(Fecha, PT, FR,ES)), columns=['Fecha','Precio_PT','precio_FR','precio_ES'])

In [21]:
df_Precio_luz_diario.head()

,Fecha,Precio_PT,precio_FR,precio_ES
0,2016-01-01,26.595000,22.634583,26.308333
1,2016-01-02,30.099583,21.922917,29.030833
2,2016-01-03,21.298750,17.063333,17.601667
3,2016-01-04,30.398750,26.979583,29.467500
4,2016-01-05,30.437500,32.502500,30.437500


## Precio del gas para cada día

In [163]:
# Para entrar a la pagina web deseada hay que usar un HEADER
HEADER = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

In [165]:
navegador = webdriver.Chrome(service=driver_service)

In [167]:
navegador.get('https://www.fxempire.es/commodities/natural-gas/history')
soup = BeautifulSoup(navegador.page_source)

In [173]:
# Dentro de la ventana emergente hay que seleccionar las fechas interesadas y luego volver a ejecutar
soup = BeautifulSoup(navegador.page_source)

In [175]:
titulos_gas=[]
titulos_gas.append(soup.find_all('th')[0].text)
titulos_gas.append(soup.find_all('th')[1].text)
titulos_gas.append(soup.find_all('th')[2].text)
titulos_gas.append(soup.find_all('th')[4].text)
titulos_gas

['Fecha ', 'Máximo ', 'Mínimo ', 'Cierre ']

In [177]:
lista=soup.find_all('td', attrs={"class":"Td-sc-do7dui-0"})[:-5]
len(lista)

15834

In [179]:
fecha=[]
maximo=[]
minimo=[]
precio_gas=[]
for i in range ( 0,2639 ):
    fecha.append(lista[6*i].text)
    maximo.append(lista[6*i+1].text)
    minimo.append(lista[6*i+2].text)
    precio_gas.append(lista[6*i+4].text)

In [181]:
df_gas = pd.DataFrame(list(zip(fecha, maximo, minimo,precio_gas)), columns=titulos_gas)

In [ ]:
# Desafortunadamente no hay datos de todos los días, asi que hay que interpolar para los días que faltan

In [ ]:
df_gas.columns = df_gas.columns.str.strip()
df_aux=pd.merge(df_balance, df_gas, on='Fecha', how='outer')
df_Precio_gas=df_aux[['Fecha','Máximo','Mínimo','Cierre']]

# Interpolación de los datos
df_Precio_gas.Máximo.interpolate(method='linear', inplace=True, limit_direction="both")

df_Precio_gas.to_csv('Precio_gas.csv', index=False)

## Temperatura máxima, mínima y la medía para 10 ciudades y un valor aproximado para España

In [ ]:
# Desde AEMET se han obtenido direcatemente las temperaturas máximas y minimas diarias de 10 ciudades de España

In [ ]:
ciudades=['Madrid','Barcelona','Bilbo','Valencia','Sevilla','A_Coruña','Zaragoza','Las_Palmas','Valladolid','Malaga']

In [ ]:
# Para cada día se ha calculado una tempereatura media
# Después se ha hecho una media aritmetica dandole a cada ciudad un peso distinto dependiendo del numero de habitantes para calcular una temperatura media para España

# Dataset definitivo

In [31]:
def juntar_todo():
    ''' Esta función junta todos los dataset obtenidos en uno solo
    '''
    df_Precio_Luz_dia=pd.read_csv('Precio_Luz_dia.csv')
    df_Balance=pd.read_csv('Balance_diario.csv')
    df_gas=pd.read_csv('Precio_gas.csv')
    df_T=pd.read_csv('Temperaturas.csv')
    
    dfs=[df_Balance,df_T,df_gas,df_Precio_Luz_dia]
    
    df = dfs[0]
    for d in dfs[1:]:
        df = df.merge(d, on='Fecha', how='inner')
    df['Fecha']=pd.to_datetime(df['Fecha'])
    return df

In [37]:
juntar_todo().head()

,Fecha,Hidráulica,Eólica,Solar fotovoltaica,Solar térmica,Hidroeólica,Otras renovables,Residuos renovables,Generación renovable,Turbinación bombeo,...,T_Media_Malaga,T_Max_España,T_Min_España,T_Media_España,Máximo,Mínimo,Cierre,Precio_PT,precio_FR,precio_ES
0,2016-01-01,31.642298,202.916119,9.021566,0.695552,0.011714,7.543142,1.754805,253.585196,9.886914,...,16.15,15.8058,9.5259,12.66585,2.278,2.241,2.246,26.595000,22.634583,26.308333
1,2016-01-02,35.554520,235.804590,12.635816,1.260093,0.013572,7.876332,1.613110,294.758033,24.261468,...,16.60,14.9438,8.3085,11.62615,2.278,2.241,2.246,30.099583,21.922917,29.030833
2,2016-01-03,35.920392,311.377459,5.096597,0.095938,0.020916,7.565244,1.662908,361.739455,12.502102,...,17.35,15.0080,7.9229,11.46545,2.278,2.241,2.246,21.298750,17.063333,17.601667
3,2016-01-04,65.278775,258.510870,4.849798,0.069916,0.028387,7.953584,2.133319,338.824649,30.484420,...,19.45,15.7991,10.5322,13.16565,2.287,2.156,2.206,30.398750,26.979583,29.467500
4,2016-01-05,79.728796,290.893912,12.562126,1.432872,0.033797,7.944637,2.474560,395.070700,20.729332,...,14.60,13.1176,7.1642,10.14090,2.267,2.176,2.256,30.437500,32.502500,30.437500
